# **Importação das Bibliotecas**

In [ ]:
import numpy as np
import nltk
nltk.download('stopwords')
import os # Novo: para interagir com o sistema de arquivos
import zipfile # Novo: para manipular arquivos zip
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Pontos Turisticos**

In [ ]:
# 1.1 Comando para fazer o upload do arquivo 'pontos_turisticos.zip'
# Execute esta célula e selecione seu arquivo no pop-up.

from google.colab import files
uploaded = files.upload()

Saving pontos_turisticos.zip to pontos_turisticos.zip


In [ ]:
# 1.2 Extrair o conteúdo do zip

zip_filename = 'pontos_turisticos.zip'
if zip_filename in uploaded:
  with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
  print(f"Arquivos extraídos para o diretório atual")
else:
    print("ERRO: O arquivo zip não foi encontrado após o upload.")
# ---------------------------------------------



Arquivos extraídos para o diretório atual


In [ ]:
# --- 2. CARREGAR OS ARQUIVOS ESTRUTURALMENTE ---

descricao = '''Estou procurando uma praia bem popular e movimentada,
que tenha uma faixa de areia grande onde eu possa caminhar. Preciso de um lugar com muita
infraestrutura na orla, como quiosques e restaurantes por perto, e que seja segura para família e
crianças. O ideal é que as águas sejam calmas e rasas para tomar banho,
mas que a região tenha um clima animado e agitado.'''

# Dicionário para armazenar o nome do ponto e o seu texto
pontos_dados = {}
# Diretório onde os arquivos foram extraídos
diretorio_textos = 'pontos_turisticos'

# Listar todos os arquivos .txt na pasta
for nome_arquivo in os.listdir(diretorio_textos):
    if nome_arquivo.endswith('.txt'):
        caminho_completo = os.path.join(diretorio_textos, nome_arquivo)

        # O nome do ponto turístico será o nome do arquivo (sem a extensão .txt)
        nome_ponto = nome_arquivo.replace('.txt', '').replace('_', ' ') # Remove .txt e substitui _ por espaço

        # Ler o conteúdo do arquivo
        with open(caminho_completo, "r", encoding="utf-8") as f:
            conteudo = f.read()

        pontos_dados[nome_ponto] = conteudo

#porm meio das chaves e valores, temos:
nomes_pontos = list(pontos_dados.keys()) # Lista de nomes
textos_pontos = list(pontos_dados.values()) # Lista de textos

compare_list = [descricao] + textos_pontos #a descrição do usuário e os textos dos pontos turísticos



# **Stopwords**

In [ ]:
#lista com as stopwords do português

from nltk.corpus import stopwords

# 1. Pega a lista padrão de stopwords em português do NLTK
nltk_stop = set(stopwords.words('portuguese'))

custom_stop = [
    # Palavras comuns que NÃO agregam valor de busca
    'local', 'conhecer', 'visita', 'visitantes', 'arredores', 'visto', 'conta',
    'pode', 'podem', 'este', 'eu', 'ser', 'seja', 'pela', 'como', 'sendo',
    'km', '500',

    # Verbos conjugados e pronomes de busca que o NLTK pode ter omitido
    'estou', 'procurando', 'tenha', # do texto da descrição
    'onde', 'ter', 'possa', 'preciso', 'ideal', 'seja', 'para' ]

# 3. Combina as duas listas para o vetorizador
br_stop = list(nltk_stop.union(set(custom_stop)))






# **Vetorização do texto**

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=br_stop) #Remover as stop words

#vetorização dos textos
tf_idf_matrix = tfidf_vectorizer.fit_transform(compare_list)



# **Impressão das similaridades**

In [ ]:
#similaridade dos vetores da descrição e dos pontos
similaridades = cosine_similarity(tf_idf_matrix[0:1], tf_idf_matrix[1:]).flatten() #flatten transforma o resultado em uma lista simples

#veremos cada similaridade correspondente a cada nome dos pontos e seus angulos
for i, sim in enumerate(similaridades):
  nome = nomes_pontos[i]
  angulo = np.degrees(np.arccos(sim)) #arccos retorna angulo em radianos, degrees retorna eles em graus
  display(f"{nome}: Sim = {sim:.3f} | Ângulo = {angulo:.2f} graus")

'Monte Serrat: Sim = 0.000 | Ângulo = 90.00 graus'

'Praia enseada: Sim = 0.229 | Ângulo = 76.74 graus'

'Praia de Pitangueiras: Sim = 0.086 | Ângulo = 85.08 graus'

'Museu do Cafe: Sim = 0.000 | Ângulo = 90.00 graus'

'Aquário Municipal de Santos: Sim = 0.000 | Ângulo = 90.00 graus'

'Praia do Gonzaga: Sim = 0.129 | Ângulo = 82.57 graus'

'Basílica de Santo Antônio de Pádua: Sim = 0.000 | Ângulo = 90.00 graus'

'Museu pele: Sim = 0.027 | Ângulo = 88.46 graus'

# **Ponto Turistico Recomendado**

In [ ]:
id_mais_parecido = similaridades.argmax() #argmax pega o indice que possui a maior similaridade
nome_mais_parecido = nomes_pontos[id_mais_parecido] #utiliza o id mais parecido para achar o nome
texto_mais_parecido = textos_pontos[id_mais_parecido]

print("=" * 50)

simi_max = similaridades[id_mais_parecido]
ang = np.degrees(np.arccos(simi_max))
print(f'✅{nome_mais_parecido} é o ponto turístico mais similar à sua descrição com {ang:.2f} graus ')
display(texto_mais_parecido)

print("=" * 50)


✅Praia enseada é o ponto turístico mais similar à sua descrição com 76.74 graus 


'A praia mais extensa do Guarujá, com cerca de 6 km de comprimento. É famosa por suas águas calmas e rasas, o que a torna perfeita para famílias com crianças e para a prática de esportes náuticos como stand up paddle e caiaque. A orla é repleta de hotéis, restaurantes e quiosques com boa infraestrutura, sendo um dos pontos mais movimentados da cidade.\n\n'